In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

class StockPricePredictor:
    def __init__(self, ticker, start_date, end_date, sequence_length=10):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.sequence_length = sequence_length
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
    
    def download_data(self):
        data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        data = data[['Close']].dropna()
        return data
    
    def preprocess_data(self, data):
        scaled_data = self.scaler.fit_transform(data)
        return scaled_data
    
    def create_sequences(self, data):
        X, y = [], []
        for i in range(len(data) - self.sequence_length):
            X.append(data[i:(i + self.sequence_length), 0])
            y.append(data[i + self.sequence_length, 0])
        return np.array(X), np.array(y)
    
    def build_model(self):
        self.model = Sequential([
            LSTM(units=50, return_sequences=True, input_shape=(self.sequence_length, 1)),
            LSTM(units=50),
            Dense(units=1)
        ])
        self.model.compile(optimizer='adam', loss='mean_squared_error')
    
    def train_model(self, X_train, y_train, epochs=100, batch_size=32):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    def evaluate_model(self, X_train, y_train, X_test, y_test):
        train_loss = self.model.evaluate(X_train, y_train, verbose=0)
        test_loss = self.model.evaluate(X_test, y_test, verbose=0)
        print("Train Loss:", train_loss)
        print("Test Loss:", test_loss)
    
    def predict_future(self, data):
        X_forecast, _ = self.create_sequences(data)
        X_forecast = X_forecast.reshape(X_forecast.shape[0], X_forecast.shape[1], 1)
        predictions = self.model.predict(X_forecast)
        predictions = self.scaler.inverse_transform(predictions)
        return predictions.flatten()